# Tensorflow 설치

### Python install

* Python 3.5.x 이상 - 64 bit 버전으로 설치해야 한다.
* [다운로드](https://www.python.org/downloads/release/python-352/)

### Install with Anaconda

https://www.tensorflow.org/install/install_windows

### Install with pip

* python -m pip install --upgrade pip

* pip3 install --upgrade tensorflow

# Tensorflow 기본 개념

## Tensorflow 라이브러리 구성요소

* Tensorflow : 딥러닝 모델을 정의하고, 데이터를 학습시키고, 실무에 적용하기 위해 export할 수 있는 API
* TensorBoard : Tensorflow 라이브러리에 포함된 그래프 시각화 소프트웨어. 텐서플로 그래프에서 export된 파일을 읽어서 모델의 동작을 살펴볼 수 있게 해준다.
* Tensorflow Serving : 학습된 텐서플로 모델을 실제 사용자들이 활용할 수 있게 해주는 소프트웨어.

## Data Flow Graph

Data Flow Graph는 수학 계산을 나타내는 Node와 데이터의 흐름을 나타내는 Edge를 사용한 방향성 그래프(Directed Graph)를 말한다.

![gif](./images/dataflowgraph.gif)

(이미지 출처 - https://camo.githubusercontent.com)

## 기본 개념

### Node

그래프 안에서 데이터에 적용될 계산이나 액션을 나타낸다.

### Edge

노드들 사이에서 전달되는 실제 값이다.

### Tensor

간단히 말해서 n 차원의 매트릭스이다. 노드는 데이터를 텐서로 주고 받는다.

* 0-D Tensor = scalar
* 1-D Tensor = vector / list
* 2-D Tensor = matrix
* n-D Tensor

### 의존성 (Dependencies)

* 한 노드는 연결된 이전 노드에 의존성을 가지고 있다. 즉, 이전 노드의 계산 결과를 필요로 한다.
* cycle을 가진 그래프는 순환 의존성 때문에 텐서플로로 표현할 수 없다. (무한 루프)
* 유한한 갯수의 그래프를 복사하여 입출력을 연결하여 순환 의존성을 흉내낼 수 있다.

## 간단한 그래프 예제

![간단한그래프예제](./images/tf01.jpg)

* input node를 사용한다.
* mul 노드와 add 노드 중 어느 것이 먼저 수행되는지 알 수 없다. 실제로는 서로 무관하게 독립적으로 실행된다고 할 수 있다.

In [1]:
import tensorflow as tf

In [2]:
a = tf.constant(5, name='input_a')
b = tf.constant(3, name='input_b')
c = tf.multiply(a,b, name='mul_c')  # operartion
d = tf.add(a,b, name='add_d')
e = tf.add(c,d, name='add_e')

In [3]:
sess = tf.Session()   # 그래프를 실행하기 위한 인터페이스

In [4]:
writer = tf.summary.FileWriter("c:\Github\output", sess.graph)
print(sess.run(e))
writer.close()

23


#### tensorboard로 그래프 조회

* [참조 블로그](http://learningtensorflow.com/Visualisation/)

In [14]:
writer = tf.summary.FileWriter('c:\Github\output', graph=sess.graph)  # log dir

터미널에서 다음 명령 실행

tensorboard --logdir="c:\Github\output"

![img](./images/tensorboard_terminal.jpg)

웹브라우저에서 다음 url 입력 > GRAPHS 메뉴로 이동

http://localhost:6006

![img](./images/tensorboard01.jpg)

## Graph

텐서플로는 라이브러리가 로딩될 때 자동으로 graph를 생성하고 이를 default로 할당한다. 그래서 명시하지 않아도 operation 이나 tensor들은 자동으로 디폴트 그래프에 할당된다.

In [1]:
# 디폴트 그래프 핸들링이 필요한 경우

In [3]:
default_graph = tf.get_default_graph()

텐서플로에서는 대부분 디폴트 그래프만을 다룬다. 하지만 서로 관련이 없는 여러 개의 모델을 사용할 경우 다수의 그래프를 만들 필요가 있다.

In [4]:
# 여러 개의 그래프를 사용하는 방법 1

In [ ]:
g1 = tf.Graph()
g2 = tf.Graph()

with g1.as_default():
    # define g1 operations, tensors...
    
with g2.as_default():
    # define g2 operations, tensors...

In [ ]:
# 여러 개의 그래프를 사용하는 방법 2

In [ ]:
g1 = tf.get_default_graph()
g2 = tf.Graph()

with g1.as_default():
    # define g1 operations, tensors...
    
with g2.as_default():
    # define g2 operations, tensors...

In [5]:
# graph reset
tf.reset_default_graph()

## Session

그래프의 실행을 담당한다. 3개의 optional parameter를 가진다.

* target : 사용할 실행 엔진 지정. 분산환경에서 세션을 사용할 경우 필요.
* graph : 실행될 그래프 객체를 지정. 여러 개의 그래프를 사용하는 경우 필요.
* config : 사용할 cpu, gup 개수 등 옵션 지정.

FEED dictionary
* 그래프에서 텐서 값을 override. 파이썬의 딕셔너리 객체 사용.

In [6]:
import tensorflow as tf

In [5]:
a = tf.add(3, 6)
b = tf.multiply(a, 5)

sess = tf.Session()
sess.run(b)

45

In [6]:
replace_dict = {a: 15}
sess.run(b, feed_dict=replace_dict)

75

In [7]:
sess.close()

In [ ]:
# Session을 컨텍스트 매니저로 사용하면 코드가 범위를 벗어날 때 session이 자동으로 close 된다.

In [ ]:
with tf.Session() as sess:
    # run graph....
    
# session close automatically.

## Placeholder

placeholder는 텐서 객체인 것처럼 행동하지만 생성될 때 값을 갖지 않는다. 실행시에 입력될 텐서를 위한 input 노드로서 동작한다.

In [8]:
import tensorflow as tf
import numpy as np

In [9]:
# int32 데이터 타입의 길이가 2인 placeholder 생성
a = tf.placeholder(tf.int32, shape=[2])

In [10]:
b = tf.reduce_prod(a)
c = tf.reduce_sum(a)

# finish
d = tf.add(b, c)

In [11]:
sess = tf.Session()

In [12]:
input_dict = {a: np.array([5,3], dtype=np.int32)}

In [13]:
sess.run(d, feed_dict=input_dict)

23

## 변수

In [ ]:
# 변수 설정

In [29]:
my_var = tf.Variable(7)

변수의 초기값은 일반적으로 0, 1 또는 랜덤값으로 이루어진 large tensor이므로, 이를 생성하기 쉽게 다양한 operation을 제공한다.

In [19]:
zeros = tf.zeros([3,3])   # 0으로 이루어진 3x3 matrix

In [20]:
ones = tf.ones([6])   # 1로 이루어진 6자리 vector

In [16]:
unis = tf.random_uniform([3,3,3], minval=0, maxval=20)
unis

<tf.Tensor 'random_uniform:0' shape=(3, 3, 3) dtype=float32>

In [21]:
normal = tf.random_normal([3,3,3], mean=0.0, stddev=1.0)
normal

<tf.Tensor 'random_normal:0' shape=(3, 3, 3) dtype=float32>

In [22]:
tnormal = tf.truncated_normal([3,3,3], mean=0.0, stddev=1.0)   # 평균에서 표준편차 2배 이상인 값은 생성하지 않는다.

In [23]:
rand_var = tf.Variable(tf.truncated_normal([2,2]))

변수의 초기화

변수 객체는 그래프 내부에 존재하지만, 변수의 상태는 Session에 의해 관리된다. 그래서 변수는 Session 내부에서 초기화해야 Session이 변수의 값들을 추적한다.

In [27]:
init = tf.global_variables_initializer()   # initialize_all_variables - deprecated 
sess = tf.Session()
sess.run(init)

In [31]:
# 특정 변수만 초기화
init2 = tf.variables_initializer([my_var])

변수 값 변경

In [32]:
var1 = tf.Variable(2)

In [33]:
var2 = var1.assign(var1 * 5)   # 변수에 새로운 값 할당

In [34]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [36]:
sess.run(var2)

10

In [37]:
sess.run(var2)

50

In [38]:
sess.run(var1.assign_add(1))   # 1 씩 증가

51

In [39]:
sess.run(var1.assign_sub(2))   # 2 씩 감소

49

In [40]:
sess.run(init)   # 변수의 값을 초기값으로 reset
sess.run(var1)

2

Optimizer는 학습을 진행하면서 변수의 값을 수정한다. 만약 변수의 값이 자동으로 변경되면 않도록 하려면 trainable 옵션을 false로 주면 된다.

In [41]:
constant_var = tf.Variable(0, trainable=False)